<a href="https://colab.research.google.com/github/mikful/dlai-tf-in-practice/blob/master/TensorFlow_in_Practice_Course_3_Week_3_Exercise_Sentiment140_with_GloVe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
import json
import csv
import random
import numpy as np

%tensorflow_version 2.x
import tensorflow as tf
print(f'TensorFlow version: {tf.__version__}')

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers


embedding_dim = 100
max_length = 16
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size= 160000
test_portion=.1

corpus = []


TensorFlow version: 2.1.0


In [28]:

# Note that I cleaned the Stanford dataset to remove LATIN1 encoding to make it easier for Python CSV reader
# You can do that yourself with:
# iconv -f LATIN1 -t UTF8 training.1600000.processed.noemoticon.csv -o training_cleaned.csv
# I then hosted it on my site to make it easier to use in this notebook

!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/training_cleaned.csv \
    -O /tmp/training_cleaned.csv

--2020-02-13 16:24:54--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/training_cleaned.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 66.102.1.128, 2a00:1450:400c:c03::80
Connecting to storage.googleapis.com (storage.googleapis.com)|66.102.1.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 238942690 (228M) [application/octet-stream]
Saving to: ‘/tmp/training_cleaned.csv’

/tmp/training_clean 100%[===================>] 227.87M  36.8MB/s    in 6.2s    

2020-02-13 16:25:01 (36.8 MB/s) - ‘/tmp/training_cleaned.csv’ saved [238942690/238942690]



In [48]:
#Option 1 Solution for creating the corpus from csv using Python csv
%%time
num_sentences = 0
corpus = []

with open("/tmp/training_cleaned.csv") as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
      # Your Code here. Create list items where the first item is the text, found in row[5], and the second is the label. 
      # Note that the label is a '0' or a '4' in the text. When it's the former, make
      # your label to be 0, otherwise 1. Keep a count of the number of sentences in num_sentences
        list_item=[]
        if row[0] == '0':
          label = 0
        else:
          label = 1
        list_item = [row[5], label]
        num_sentences = num_sentences + 1
        corpus.append(list_item)


CPU times: user 5.35 s, sys: 84.7 ms, total: 5.44 s
Wall time: 5.45 s


In [0]:
#Option 2 Solution for creating the corpus from csv using Pandas - much slower!!!

# %%time
# num_sentences = 0

# import pandas as pd
# df = pd.read_csv("/tmp/training_cleaned.csv", header=None) # read csv in pandas dataframe for ease of use
# #df.head()

# for index, row in df.iterrows():
#   if row[0] == 0:
#     label = 0
#   elif row[0] == 4:
#     label = 1
#   list_item = [row[5], label]
#   num_sentences = num_sentences + 1
#   #print(row[5], label)
#   corpus.append(list_item)


In [49]:
print(num_sentences)
print(len(corpus))
print(corpus[1])

# Expected Output:
# 1600000
# 1600000
# ["is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!", 0]

1600000
1600000
["is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!", 0]


In [0]:
# Shuffle into training set using training_size parameter set above

sentences=[]
labels=[]
random.shuffle(corpus)
for x in range(training_size):
    sentences.append(corpus[x][0])
    labels.append(corpus[x][1])

# Create word index from tokenizer on training set

tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)

word_index = tokenizer.word_index
vocab_size=len(word_index)

# Tokenize the training sentences and pad, truncate

sequences = tokenizer.texts_to_sequences(sentences)
padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

# Split training set sentences and labels into train/test sets

split = int(test_portion * training_size)

test_sequences = padded[:split]
training_sequences = padded[split:training_size]
test_labels = np.array(labels[:split])
training_labels = np.array(labels[split:training_size])

In [56]:
print(vocab_size)
print(word_index['i'])
# Expected Output
# 138858
# 1

138498
1


In [57]:
# Create the word embeddings using GloVe

# Note this is the 100 dimension version of GloVe from Stanford
# I unzipped and hosted it on my site to make this notebook easier
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/glove.6B.100d.txt \
    -O /tmp/glove.6B.100d.txt
embeddings_index = {};
with open('/tmp/glove.6B.100d.txt') as f:
    for line in f:
        values = line.split();
        word = values[0];
        coefs = np.asarray(values[1:], dtype='float32');
        embeddings_index[word] = coefs;

embeddings_matrix = np.zeros((vocab_size+1, embedding_dim));
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word);
    if embedding_vector is not None:
        embeddings_matrix[i] = embedding_vector;

--2020-02-13 16:37:32--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/glove.6B.100d.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.76.128, 2a00:1450:400c:c08::80
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.76.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 347116733 (331M) [text/plain]
Saving to: ‘/tmp/glove.6B.100d.txt’

/tmp/glove.6B.100d. 100%[===================>] 331.04M   122MB/s    in 2.7s    

2020-02-13 16:37:35 (122 MB/s) - ‘/tmp/glove.6B.100d.txt’ saved [347116733/347116733]



In [58]:
print(len(embeddings_matrix))
# Expected Output
# 138859

138499


In [68]:
# Create the model

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size+1, embedding_dim, input_length=max_length, weights=[embeddings_matrix], trainable=False),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 16, 100)           13849900  
_________________________________________________________________
bidirectional_9 (Bidirection (None, 16, 256)           234496    
_________________________________________________________________
bidirectional_10 (Bidirectio (None, 128)               164352    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 24)                3096      
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 25        
Total params: 14,251,869
Trainable params: 401,969
Non-trainable params: 13,849,900
____________________________________

In [69]:
# Train 

num_epochs = 10
history = model.fit(training_sequences, 
                    training_labels, 
                    epochs=num_epochs, 
                    validation_data=(test_sequences, test_labels))

print("Training Complete")


Train on 144000 samples, validate on 16000 samples
Epoch 1/10
144000/144000 [==============================] - 58s 403us/sample - loss: 0.5312 - accuracy: 0.7296 - val_loss: 0.4903 - val_accuracy: 0.7591
Epoch 2/10
144000/144000 [==============================] - 54s 377us/sample - loss: 0.4770 - accuracy: 0.7685 - val_loss: 0.4668 - val_accuracy: 0.7729
Epoch 3/10
144000/144000 [==============================] - 56s 391us/sample - loss: 0.4465 - accuracy: 0.7869 - val_loss: 0.4642 - val_accuracy: 0.7763
Epoch 4/10
144000/144000 [==============================] - 56s 392us/sample - loss: 0.4182 - accuracy: 0.8051 - val_loss: 0.4572 - val_accuracy: 0.7856
Epoch 5/10
144000/144000 [==============================] - 57s 394us/sample - loss: 0.3867 - accuracy: 0.8229 - val_loss: 0.4795 - val_accuracy: 0.7729
Epoch 6/10
 64352/144000 [============>.................] - ETA: 30s - loss: 0.3444 - accuracy: 0.8441

KeyboardInterrupt: ignored

In [0]:
import matplotlib.image  as mpimg
import matplotlib.pyplot as plt

#-----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------
acc=history.history['acc']
val_acc=history.history['val_acc']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(len(acc)) # Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot(epochs, acc, 'r')
plt.plot(epochs, val_acc, 'b')
plt.title('Training and validation accuracy')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend(["Accuracy", "Validation Accuracy"])

plt.figure()

#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------
plt.plot(epochs, loss, 'r')
plt.plot(epochs, val_loss, 'b')
plt.title('Training and validation loss')
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend(["Loss", "Validation Loss"])

plt.figure()


# Expected Output
# A chart where the validation loss does not increase sharply!